In [7]:
#library('caTools')
library('caret')

In [8]:
full_data <- read.csv("movies_allfeaturesfinal.csv")
# Select only required columns
columns_needed <- c('original_language', 'popularity', 'release_date', 'vote_average', 'tmdb_genre', 'revenue', 
                    'runtime', 'Rating','Votes', 'Certificates', 'R', 'G', 'B', 'woman', 'director', 'murder', 
                    'short', 'film', 'relationship', 'novel', 'alien', 'horror', 'based', 'space', 'war', 
                    'death', 'zombie', 'ghost', 'police', 'movie', 'nudity', 'love', 'time', 'sequel', 
                    'dystopia', 'killer', 'comic', 'drama', 'biography', 'monster', 'violence', 'family', 
                    'female', 'thriller', 'gay', 'book', 'post', 'supernatural', 'future', 'suspense', 'found', 
                    'superhero', 'footage', 'brightness_level', 'genre_category')
data <- full_data[columns_needed]
#data
str(data)

#Convert runtime from charecter type to Integer
data$runtime <- as.numeric(data$runtime)
data$brightness_level <- as.numeric(data$brightness_level)

# Convert original language from factor to charecter
data$original_language <- as.character(data$original_language)


#Convert release date from factor to date and then extract year out of that and convert to integer
data$release_year <- as.numeric(format(as.Date(data$release_date, format = "%m/%d/%Y"), "%Y"))
# Filter out movies released starting 2017
data <- data[data$release_year < 2017, ]
cat("\nConsidering movies till 2016 only\n")
str(data)

data$genre <- data$genre_category

# Final list of columns, dropping unwanted columns used in filtering data
final_columns <- c('genre', 'popularity', 'release_year', 'vote_average', 'revenue', 'runtime', 'Rating', 'Votes',
                   'woman', 'director', 'murder', 'short', 'film', 'relationship', 'novel', 'alien', 
                   'horror', 'based', 'space', 'war', 'death', 'zombie', 'ghost', 'police', 'movie', 'nudity', 
                   'love', 'time', 'sequel', 'dystopia', 'killer', 'comic', 'drama', 'biography', 'monster', 
                   'violence', 'family', 'female', 'thriller', 'gay', 'book', 'post', 'supernatural', 'future', 
                   'suspense', 'found', 'superhero', 'footage', 'brightness_level')

'data.frame':	5695 obs. of  55 variables:
 $ original_language: Factor w/ 63 levels "","ar","bg","bn",..: 12 12 12 12 12 12 12 12 12 12 ...
 $ popularity       : num  42.68 20.11 7.47 6.15 4.83 ...
 $ release_date     : Factor w/ 1666 levels "1/1/2011","1/1/2012",..: 48 764 560 998 128 587 978 1655 729 1255 ...
 $ vote_average     : num  6.8 6 4.7 0 5.4 5.5 6.5 0 7.1 0 ...
 $ tmdb_genre       : Factor w/ 1262 levels "[Action, Adventure, Animation, Comedy, Crime]",..: 801 790 805 234 801 569 805 805 177 34 ...
 $ revenue          : int  269961806 51998634 0 0 0 24209725 0 0 0 0 ...
 $ runtime          : Factor w/ 195 levels "0","1","10","100",..: 22 7 6 1 191 54 185 1 182 1 ...
 $ Rating           : num  7.4 7 4.5 0 4 6.5 6.2 0 6.5 0 ...
 $ Votes            : num  110172 32733 9528 0 4457 ...
 $ Certificates     : Factor w/ 2058 levels "","Argentina:13",..: 201 213 63 1 123 224 1019 1 1705 1 ...
 $ R                : num  49 8 22 0 21 161 42 21 193 31 ...
 $ G                : num  15 1

In [9]:
table(data$genre)


AcAdDr AcCrTh AcThMi CoFaDr     Ho     Ot RoCoDa   ScFa 
   408   1512    609     46    516     21   1108    278 

In [10]:
# Creating a dataset of required columns only
unfiltered_data <- data[final_columns]
#str(unfiltered_data)

set.seed(101) # Set Seed so that same sample can be reproduced in future also
# Now Selecting 75% of data as sample from total 'n' rows of the data  
#uf_sample <- sample.int(n = nrow(unfiltered_data), size = floor(.75*nrow(unfiltered_data)), replace = F)
#uf_sample <- sample.split(unfiltered_data, SplitRatio = 3/4)

# Creating Train and Test datasets
uf_sample <- createDataPartition(y=unfiltered_data$genre, p=0.75, list=FALSE)
unfiltered_train_data <- unfiltered_data[uf_sample, ]
unfiltered_test_data  <- unfiltered_data[-uf_sample, ]

#unfiltered_train_data <- unfiltered_data[sample(nrow(unfiltered_data), as.integer(nrow(unfiltered_data)*.8)), ]
#unfiltered_test_data <- unfiltered_data[!duplicated(rbind(unfiltered_train_data, unfiltered_data))[-seq_len(nrow(unfiltered_train_data))], ]

# Writing data to files to distribute between teams
write.csv(unfiltered_train_data, file = "unfiltered_train_data.csv",row.names=FALSE)
write.csv(unfiltered_test_data, file = "unfiltered_test_data.csv",row.names=FALSE)

In [11]:
table(unfiltered_train_data$genre)


AcAdDr AcCrTh AcThMi CoFaDr     Ho     Ot RoCoDa   ScFa 
   306   1134    457     35    387     16    831    209 

In [12]:
# Filtering Englis movies only 
data <- data[data$original_language == 'en', ]
cat("\nAfter filtering English Movies only\n")
#str(data)

# Filterout movies whose runtime is less than 60 mins
data <- data[data$runtime >= 60, ]
cat("\nConsidering movies with atleast 60 min runtime\n")
#str(data)

table(data$genre)

# Select only needed columns
filtered_data <- data[final_columns]
#str(filtered_data)

set.seed(101) # Set Seed so that same sample can be reproduced in future also
# Now Selecting 75% of data as sample from total 'n' rows of the data  
#f_sample <- sample.int(n = nrow(filtered_data), size = floor(.75*nrow(filtered_data)), replace = F)
#f_sample <- sample.split(filtered_data, SplitRatio = 3/4)

# Creating Train and Test datasets
f_sample <- createDataPartition(y=filtered_data$genre, p=0.75, list=FALSE)
filtered_train_data <- filtered_data[f_sample, ]
filtered_test_data  <- filtered_data[-f_sample, ]

#filtered_train_data <- filtered_data[sample(nrow(filtered_data), as.integer(nrow(filtered_data)*.8)), ]
#filtered_test_data <- filtered_data[!duplicated(rbind(filtered_train_data, filtered_data))[-seq_len(nrow(filtered_train_data))], ]

# Writing data to files to distribute between teams
write.csv(filtered_train_data, file = "filtered_train_data.csv",row.names=FALSE)
write.csv(filtered_test_data, file = "filtered_test_data.csv",row.names=FALSE)

table(filtered_train_data$genre)
#all_cert <- unlist(strsplit(as.character(data$Certificates), ','))
#str(all_cert)
#usa_cert <- temp[grepl("^USA:*", temp)]
#str(usa_cert)
#data$Certificates <- (colsplit(usa_cert, split = ':', names = c('Country','Cert','x','y'))[,2])


After filtering English Movies only

Considering movies with atleast 60 min runtime



AcAdDr AcCrTh AcThMi CoFaDr     Ho     Ot RoCoDa   ScFa 
   128    738    285     27    313      8    441    148 


AcAdDr AcCrTh AcThMi CoFaDr     Ho     Ot RoCoDa   ScFa 
    96    554    214     21    235      6    331    111 